# Notebook, to be executed by mobu, to test fastapi-bootcamp

## Get an HTTP client for the service

In [ ]:
# RSPClient is in "recommended" as of 09 May 2024, and in w_2024_19 and later
# For earlier images, you'll need to construct your own client.

from lsst.rsp import RSPClient

http_client = RSPClient("/fastapi-bootcamp")

## Test application endpoints

### Get JSON metadata

In [ ]:
resp = (await http_client.get("/")).json()
assert resp["metadata"]["repository_url"] == "https://github.com/lsst-sqre/fastapi-bootcamp"

### Get a basic greeting

In [ ]:
resp = (await http_client.get("/hello")).json()
assert resp == "Hello, SQuaRE Services Bootcamp!"

### Get a JSON greeting in English

In [ ]:
resp = (await http_client.get("/en-greeting")).json()
assert resp["greeting"] == "Hello, SQuaRE Services Bootcamp!"

### Get a JSON Greeting in French

In [ ]:
resp = (await http_client.get("/greeting", params={"language": "fr"})).json()
assert resp["greeting"] == "Bonjour, SQuaRE Services Bootcamp!"

### Get a JSON greeting, POST-style

In [ ]:
payload = { "name": "Mobu bot",
            "language": "es" }
resp = (await http_client.post("/greeting", json=payload)).json()
assert resp["greeting"] == "¡Hola, Mobu bot!"

### Get a JSON greeting, with the parameter in the path

In [ ]:
resp = (await http_client.get("/greeting/es")).json()
assert resp["greeting"] == "¡Hola, SQuaRE Services Bootcamp!"

### Log a message

In [ ]:
payload = { "name": "Mobu bot",
            "language": "fr" }
resp = (await http_client.post("/log-demo", json=payload)).json()
assert resp["greeting"] == "Bonjour, Mobu bot!"

### Don't test "Raise An Internal Service Exception"

Sure, we could hit this endpoint too...but since this is designed to run with Mobu, we will not do so, because we don't want our Slack channel to fill with spam.

### Demonstrate Custom Dependencies

In [ ]:
# Here, the service picks a different value each time it starts.

resp = (await http_client.get("/dependency-demo")).json()
adjective, animal = resp["persistent_value"].split()
assert adjective and animal  # Both should be truthy

### Astroplan: get all observers

In [ ]:
# This is a paginated list, so we have to make multiple calls to get all observers

page=1  # This API is one-indexed
total=0 # Total number of observatories listed

observer_list: list[str] = []

while True:
    resp = (await http_client.get(
        "/astroplan/observers/",params={"page":page}
    )).json()
    # Find out how many we're supposed to get, if we don't know yet.
    if not total:
        total = resp["pagination"]["total"]
    # Add the IDs for each observatory on this page
    for obsv in resp["data"]:
        observer_list.append(obsv["id"])
    # If we've run out of pages, we are done
    if not resp["pagination"]["next_url"]:
        break
    # We don't use next_url, because our client already knows most of the URL.
    page += 1
    
assert "gemini-north" in observer_list and len(observer_list) == total

### Astroplan: get observer by param

In [ ]:
resp = (await http_client.get("/astroplan/observers/",params={"name":"icecube"})).json()
# We get back the paginated list again, but we know the length of 'data' will be 1
obsv = resp["data"][0]
assert obsv["longitude"] == 0.0 and obsv["latitude"] == -90.0

### Astroplan: get observer by site ID

In [ ]:
from pytest import approx

resp = (await http_client.get("/astroplan/observers/rubin")).json()
assert resp["elevation"] == approx(2662.75)  # Floats aren't real!

### Astroplan: check if coordinate is observable for observer at given time

In [ ]:
payload = { "ra": "5h23m34s",
            "dec": "-69d45m22s",  # LMC
            "time": "2024-06-13T05:45:23Z"
          }
resp = (await http_client.post("/astroplan/observers/rubin/observable", json=payload)).json()
# The LMC should be visible at that time
assert resp["is_night"] and resp["above_horizon"]